In [1]:
import dores
import pandas as pd
pd.set_option('max_colwidth', -1)
pd.set_option('display.max_rows', 1000)

In [2]:
st_brieuc = dores.DiscreteEventSimulation()
entities = st_brieuc.create_entities()
st_brieuc.start_simulation()
st_brieuc.get_project_length()

7.978659217685186